In [1]:
import pulp

# Example data for m plants and n models
# Replace these with your actual data
m = 4
n = 5
b = [1737, 2646, 2690, 1253]                # capacity (hours) per plant
p = [56, 58, 45, 40, 38]                    # selling price per model
f = [42000, 100000, 35000, 31000, 23000]    # fixed cost per model
c = [15, 17, 10, 7, 4]                      # variable cost per model
t = [
    [165, 163, 188, 145, 170],
    [91, 83, 146, 190, 191],
    [151, 175, 163, 147, 77],
    [84, 188, 122, 180, 107]
]

# Create the problem
model = pulp.LpProblem("toy_manufacturing", pulp.LpMaximize)

# Decision variables
x = pulp.LpVariable.dicts("x", ((i, j) for i in range(m) for j in range(n)), 
                          lowBound=0, cat=pulp.LpInteger)
y = pulp.LpVariable.dicts("y", ((i, j) for i in range(m) for j in range(n)), 
                          cat=pulp.LpBinary)

# Objective
model += (
    pulp.lpSum(p[j] * x[(i, j)] for i in range(m) for j in range(n))
    - pulp.lpSum(f[j] * y[(i, j)] for i in range(m) for j in range(n))
    - pulp.lpSum(c[j] * x[(i, j)] for i in range(m) for j in range(n))
)

# Constraints
# Capacity constraint: sum of (1/t[i][j]) * x[i,j] <= b[i]
for i in range(m):
    model += pulp.lpSum((1 / t[i][j]) * x[(i, j)] for j in range(n)) <= b[i]

# Linking x and y: x[i,j] <= b[i] * t[i][j] * y[i,j]
for i in range(m):
    for j in range(n):
        model += x[(i, j)] <= b[i] * t[i][j] * y[(i, j)]

# Solve
model.solve(pulp.PULP_CBC_CMD(msg=0))

# Print results
print("Status:", pulp.LpStatus[model.status])
print("Objective value:", pulp.value(model.objective))
for i in range(m):
    for j in range(n):
        if pulp.value(x[(i, j)]) > 0:
            print(f"x[{i},{j}] = {pulp.value(x[(i, j)])}, y[{i},{j}] = {pulp.value(y[(i, j)])}")

Status: Optimal
Objective value: 57627803.0
x[0,0] = 286605.0, y[0,0] = 1.0
x[1,4] = 505386.0, y[1,4] = 1.0
x[2,1] = 470750.0, y[2,1] = 1.0
x[3,1] = 235564.0, y[3,1] = 1.0


In [3]:
from ortools.linear_solver import pywraplp

# Datos de ejemplo
m = 4
n = 5
b = [1737, 2646, 2690, 1253]
p = [56, 58, 45, 40, 38]
f = [42000, 100000, 35000, 31000, 23000]
c = [15, 17, 10, 7, 4]
t = [
    [165, 163, 188, 145, 170],
    [91,  83,  146, 190, 191],
    [151, 175, 163, 147, 77],
    [84,  188, 122, 180, 107]
]

# Crear solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Variables
x = {}
y = {}
for i in range(m):
    for j in range(n):
        x[(i, j)] = solver.IntVar(0, solver.infinity(), f'x[{i},{j}]')
        y[(i, j)] = solver.BoolVar(f'y[{i},{j}]')

# Función objetivo
solver.Maximize(
    solver.Sum(p[j] * x[(i, j)] for i in range(m) for j in range(n))
    - solver.Sum(f[j] * y[(i, j)] for i in range(m) for j in range(n))
    - solver.Sum(c[j] * x[(i, j)] for i in range(m) for j in range(n))
)

# Restricciones
for i in range(m):
    solver.Add(solver.Sum(x[(i, j)] / t[i][j] for j in range(n)) <= b[i])
    for j in range(n):
        solver.Add(x[(i, j)] <= b[i] * t[i][j] * y[(i, j)])

# Resolver
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print("Beneficio máximo =", solver.Objective().Value())
    print("Cantidades a producir por planta y modelo:")
    for i in range(m):
        for j in range(n):
            if x[(i, j)].solution_value() > 0:
                print(f"Planta {i}, Modelo {j}: {x[(i, j)].solution_value()}")
else:
    print("No se encontró solución óptima.")

Beneficio máximo = 57627803.0
Cantidades a producir por planta y modelo:
Planta 0, Modelo 0: 286605.0
Planta 1, Modelo 4: 505386.0
Planta 2, Modelo 1: 470750.0
Planta 3, Modelo 1: 235564.0
